# Deep Markov Model with motion model

In [1]:
from tqdm import tqdm

import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from tensorboardX import SummaryWriter
import numpy as np

In [2]:
batch_size = 128#128
epochs = 100
seed = 1
torch.manual_seed(seed)

In [3]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [4]:
#計測モデルとか
def get_ot(st,lmap,max_range):
    dis = torch.sqrt((st[:,0]-lmap[0])**2+(st[:,1]-lmap[1])**2)
    angle = torch.atan2((lmap[1]-st[:,1]),(lmap[0]-st[:,0]))-st[:,2]
    return torch.stack([dis,angle],1)
    
def get_all_ot(st,lmap,max_range):
    measure = get_ot(st,lmap[0],max_range)
    for l in range(1,len(lmap)):
        measure = torch.cat([measure, get_ot(st,lmap[l],max_range)],1)
    return torch.tensor(measure)

In [5]:
landmark_num = 10
start_pos = [2.0,4.0,0.0]#x0,y0,yaw0

In [6]:
landmark_dim = 2
x_dim = landmark_num*2
h_dim = 32 #32
hidden_dim = 32 #32
z_dim = 3
u_dim = 2
t_max = 89

In [7]:
def normalize(data, *args):
    if len(args) == 0:
        max_data = torch.max(abs(data))
        data=data/max_data
    else :
        max_data=torch.max(abs(torch.tensor(args)))
        data=data/max_data
    return data

def normalize(data, *args):
    if len(args) == 0:#未実装
        max_data = torch.max(abs(data))
        data=data/max_data
    else :
        max_data=torch.max(abs(torch.tensor(args)))
        data=data*max_data
    return data

def ot_normal(ot,range_ot):
    ot=ot.view(len(ot),10,2)
    ot[:,:,0]=normalize(ot[:,:,0],range_ot[0][0],range_ot[0][1])
    ot[:,:,1]=normalize(ot[:,:,1],range_ot[1][0],range_ot[1][1])
    ot = ot.view(len(ot),20)
    return ot

def inv_ot_normal(ot,range_ot):
    ot=ot.view(len(ot),10,2)
    ot[:,:,0]=ot[:,:,0]*torch.max(range_ot[0][0],range_ot[0][1])
    ot[:,:,1]=ot[:,:,1]*torch.max(range_ot[1][0],range_ot[1][1])
    ot = ot.view(len(ot),20)
    return ot

In [8]:
#データの読み込み
transform = transforms.Compose([transforms.ToTensor()])
kwargs = {'batch_size': batch_size, 'num_workers': 1, 'pin_memory': True}

range_ot=torch.tensor([[0.0,150.0],[-np.pi*2,np.pi*2]])#otの[dis,theta]の最大と最小
range_ut=torch.tensor([[0.0,5.0],[-np.pi/2.0,np.pi/2.0]])#utの[v,r]

#data loader #とりあえず1時系列分を分身させて食わせてる
#[time,s_x,s_y,s_yaw,uv,ur,ot[1],,,,ot[N]]
data = np.loadtxt('vehicle_motion_data.csv', delimiter=',')
data = torch.tensor([data],dtype=torch.float32)
st = data[0,:,1:(1+z_dim)]
ut = data[0,:,(1+z_dim):(1+z_dim+u_dim)]
ot = data[0,:,(1+z_dim+u_dim):(1+z_dim+u_dim+x_dim)]
t_max = len(ot)
ot=ot_normal(ot,range_ot)

ut[:,0]=normalize(ut[:,0],range_ut[0][0],range_ut[0][1])
ut[:,1]=normalize(ut[:,1],range_ut[1][0],range_ut[1][1])

print(st.size())
st=st.repeat(1000,1,1)
ut=ut.repeat(1000,1,1)
ot=ot.repeat(1000,1,1)
print(st.size())


landmark = np.loadtxt('landmark_data.csv',delimiter=',')

train = torch.utils.data.TensorDataset(ot,ut)
train_loader = torch.utils.data.DataLoader(train, shuffle=False,**kwargs)
test = torch.utils.data.TensorDataset(ot,ut)
test_loader = torch.utils.data.DataLoader(test, shuffle=False,**kwargs)

torch.Size([66, 3])
torch.Size([1000, 66, 3])


In [9]:
from pixyz.models import Model
from pixyz.losses import KullbackLeibler, CrossEntropy, IterativeLoss, StochasticReconstructionLoss
from pixyz.distributions import Bernoulli, Normal, Deterministic
from pixyz.utils import print_latex

In [23]:
class RNN(Deterministic):
    def __init__(self):
        super(RNN, self).__init__(cond_var=["x"], var=["h"])
        self.rnn = nn.GRU(x_dim, h_dim, bidirectional=True)
#         self.h0 = torch.zeros(2, batch_size, self.rnn.hidden_size).to(device)
        self.h0 = nn.Parameter(torch.zeros(2, 1, self.rnn.hidden_size))
        self.hidden_size = self.rnn.hidden_size
        
    def forward(self, x):
        h0 = self.h0.expand(2, x.size(1), self.rnn.hidden_size).contiguous()
        h, _ = self.rnn(x, h0)
        return {"h": h}

In [24]:
# class Generator(Bernoulli):
#     def __init__(self):
#         super(Generator, self).__init__(cond_var=["z"], var=["x"])
#         self.fc1 = nn.Linear(z_dim, hidden_dim)
#         self.fc2 = nn.Linear(hidden_dim, x_dim)
    
#     def forward(self, z):
#         print(z.size()) #[128,3]
#         h = F.relu(self.fc1(z))
#         return {"probs": torch.sigmoid(self.fc2(h))}
class Generator(Normal):
    def __init__(self):
        super(Generator, self).__init__(cond_var=["z"], var=["x"])
    
    def forward(self, z):#計測モデルそのまま
        ot=get_all_ot(z,landmark,[1000,1000])
        ot=ot_normal(ot,range_ot) #データを正規化しなおしてる
        return {"loc": ot,"scale":torch.tensor(0.1).to(device)}

In [25]:
class Inference(Normal):
    def __init__(self):
        super(Inference, self).__init__(cond_var=["h", "z_prev"], var=["z"])
        self.fc1 = nn.Linear(z_dim, h_dim*2)
        self.fc21 = nn.Linear(h_dim*2, z_dim)
        self.fc22 = nn.Linear(h_dim*2, z_dim)
        
    def forward(self, h, z_prev):
        h_z = torch.tanh(self.fc1(z_prev))
        h = 0.5 * (h + h_z)
        return {"loc": self.fc21(h), "scale": F.softplus(self.fc22(h))}

In [26]:
class Prior(Normal):
    def __init__(self):
        super(Prior, self).__init__(cond_var=["z_prev","u"], var=["z"])
        
    def forward(self, z_prev, u):
        # motion model for two-wheel robot x,y,orient,v,steering
        z = torch.zeros(len(z_prev),z_dim).to(device)
        uv=normalize(u[:,0],range_ut[0][0],range_ut[0][1])
        ur=normalize(u[:,1],range_ut[1][0],range_ut[1][1])
        
        z[:,2] = z_prev[:,2] + ur
        z[:,0] = z_prev[:,0] + uv * torch.cos(z_prev[:,2] + ur)
        z[:,1] = z_prev[:,1] + uv * torch.sin(z_prev[:,2] + ur)

        return {"loc": z, "scale": torch.tensor(0.3).to(device)}

In [27]:
prior = Prior().to(device)
encoder = Inference().to(device)
decoder = Generator().to(device)
rnn = RNN().to(device)

In [28]:
print(prior)
print("*"*80)
print(encoder)
print("*"*80)
print(decoder)
print("*"*80)
print(rnn)

Distribution:
  p(z|z_{prev},u)
Network architecture:
  Prior(
    name=p, distribution_name=Normal,
    var=['z'], cond_var=['z_prev', 'u'], input_var=['z_prev', 'u'], features_shape=torch.Size([])
  )
********************************************************************************
Distribution:
  p(z|h,z_{prev})
Network architecture:
  Inference(
    name=p, distribution_name=Normal,
    var=['z'], cond_var=['h', 'z_prev'], input_var=['h', 'z_prev'], features_shape=torch.Size([])
    (fc1): Linear(in_features=3, out_features=64, bias=True)
    (fc21): Linear(in_features=64, out_features=3, bias=True)
    (fc22): Linear(in_features=64, out_features=3, bias=True)
  )
********************************************************************************
Distribution:
  p(x|z)
Network architecture:
  Generator(
    name=p, distribution_name=Normal,
    var=['x'], cond_var=['z'], input_var=['z'], features_shape=torch.Size([])
  )
*****************************************************************

In [29]:
generate_from_prior = prior * decoder
print(generate_from_prior)
print_latex(generate_from_prior)

Distribution:
  p(x,z|z_{prev},u) = p(x|z)p(z|z_{prev},u)
Network architecture:
  Prior(
    name=p, distribution_name=Normal,
    var=['z'], cond_var=['z_prev', 'u'], input_var=['z_prev', 'u'], features_shape=torch.Size([])
  )
  Generator(
    name=p, distribution_name=Normal,
    var=['x'], cond_var=['z'], input_var=['z'], features_shape=torch.Size([])
  )


<IPython.core.display.Math object>

In [36]:
step_loss = CrossEntropy(encoder, decoder) + KullbackLeibler(encoder, prior)
_loss = IterativeLoss(step_loss, max_iter=t_max, 
                      series_var=["x", "h", "u"], update_value={"z": "z_prev"})
# loss = _loss
loss = _loss.expectation(rnn).mean()

In [37]:
dmm = Model(loss, distributions=[rnn, encoder, decoder, prior], 
            optimizer=optim.RMSprop, optimizer_params={"lr": 5e-4}, clip_grad_value=10)

In [38]:
print(dmm)
print_latex(dmm)

Distributions (for training): 
  p(h|x), p(z|h,z_{prev}), p(x|z), p(z|z_{prev},u) 
Loss function: 
  \sum_{t=1}^{66} - \mathbb{E}_{p(z|h,z_{prev})} \left[\log p(x|z) \right] 
Optimizer: 
  RMSprop (
  Parameter Group 0
      alpha: 0.99
      centered: False
      eps: 1e-08
      lr: 0.0005
      momentum: 0
      weight_decay: 0
  )


<IPython.core.display.Math object>

In [39]:
def data_loop(epoch, loader, model, device, train_mode=False):
    mean_loss = 0
    for idx,[o,u] in enumerate(tqdm(loader)):#batch_idx, (data, _) in enumerate(tqdm(loader)):
        o = o.to(device)
        u = u.to(device)
        batch_size = o.size()[0]
        x = o.transpose(0, 1) #多分転置してるだけ
        u = u.transpose(0, 1)
        z_prev = torch.tensor(start_pos)#初期姿勢
        z_prev = z_prev.repeat(batch_size, 1).to(device)
        if train_mode:
            mean_loss += model.train({'x': x, 'u':u, 'z_prev': z_prev}).item() * batch_size
        else:
            mean_loss += model.test({'x': x, 'u':u, 'z_prev': z_prev}).item() * batch_size
    mean_loss /= len(loader.dataset)
    if train_mode:
        print('Epoch: {} Train loss: {:.4f}'.format(epoch, mean_loss))
    else:
        print('Test loss: {:.4f}'.format(mean_loss))
    return mean_loss

In [40]:
def plot_image_from_latent(batch_size):
    x = []
    z_prev = torch.zeros(batch_size, z_dim).to(device)
    u0 = torch.zeros(batch_size, u_dim).to(device)
    for step in range(t_max):
        samples = generate_from_prior.sample({'z_prev': z_prev,'u':u0})
        x_t = decoder.sample_mean({"z": samples["z"]})
        z_prev = samples["z"]
        x.append(x_t[None, :])
    x = torch.cat(x, dim=0).transpose(0, 1)
    return x

In [41]:
writer = SummaryWriter()

history = {"train_loss":[],"test_loss":[]}

for epoch in range(1, epochs + 1):
    train_loss = data_loop(epoch, train_loader, dmm, device, train_mode=True)
    test_loss = data_loop(epoch, test_loader, dmm, device)

    writer.add_scalar('train_loss', train_loss, epoch)
    writer.add_scalar('test_loss', test_loss, epoch)

    history["train_loss"].append(train_loss)
    history["test_loss"].append(test_loss)
    
    sample = plot_image_from_latent(batch_size)[:, None][1,:]
    writer.add_image('Image_from_latent', sample, epoch)

  0%|          | 0/8 [00:00<?, ?it/s]


ValueError: Input keys are not valid, expected ['z_prev', 'h'] but got ['x', 'u', 'z_prev'].

In [ ]:
import matplotlib.pyplot as plt

plt.ylabel('$loss$', fontsize=16)
plt.xlabel('$epoch$', fontsize=16)
ay=plt.gca()
plt.title("train_loss")
plt.plot(range(epochs), [i+0.5 for i in history["train_loss"]])
plt.show()
ay=plt.gca()
plt.title("test_loss")
plt.plot(range(epochs), [i+0.4 for i in history["test_loss"]])
plt.show()

In [ ]:

inference_net = rnn*encoder
test_o = data[0,:,(1+z_dim+u_dim):(1+z_dim+u_dim+x_dim)]
test_o = torch.tensor(test_o).reshape(1,len(test_o),20).to(device)
z_prev = torch.tensor(start_pos).to(device)
infered_result = inference_net.sample({"x":test_o,"z_prev":z_prev})["z"].to("cpu")
infered_result=infered_result.numpy()

plt.plot(infered_result[:,:, 0], infered_result[:,:, 1], "co")
for i in range(len(infered_result[0])-1):
    plt.plot([infered_result[0][i][0],infered_result[0][i+1][0]],[infered_result[0][i][1],infered_result[0][i+1][1]],"r")
plt.show()